In [1]:
import pandas as pd
import numpy as np 
import os
import sys
import json

sys.path.insert(1,'/home/robin/Documents/DGS/signalement-ia/demonstrateur')
import lib_extraction

with open(os.path.abspath(os.path.join("/home/robin/Documents/DGS/data/",'Colonnes.json')), 'r') as file:
    colonnes = json.load(file)
with open(os.path.abspath(os.path.join("/home/robin/Documents/DGS/data/",'mapping.json')), 'r') as file:
    mapping = json.load(file)


In [1]:
import pandas as pd

In [2]:
data = pd.read_json('/home/robin/Documents/DGS/Annexe/L3/07_01_2020/cluster/mrv_with_clustering.json')

In [8]:
list(set(data['cluster']))[-1]

1498

In [2]:
mrv_data = pd.read_csv("/home/robin/Documents/DGS/data/declaration_mrv_complet.csv")
fabricant_liste = list(set(mrv_data["FABRICANT"].tolist()))

In [3]:
print(len(fabricant_liste))

2566


In [85]:
L = sorted(list(map(str, fabricant_liste)))

In [10]:
from gensim.utils import deaccent
import string
import re
from pandas.core.common import flatten

def clean_fabricant(text):
    STOP_WORDS = ["france","sarl",'sas','ltd','inc','sro','sa', 'ab']
    #lower
    text = text.lower()
    # Stop words
    tokens = text.split(" ") 
    tokens = [token for token in tokens if deaccent(token.lower()) not in STOP_WORDS]
    # ponctuation
    tokens = [token for token in tokens if not all(c in set(string.punctuation) for c in token.lower())]

    # Acccent
    tokens = [deaccent(token.lower()) for token in tokens]
    # Caratère spéciaux
    tokens = [re.split('[^A-Za-z]+', str(token)) for token in tokens]
    # flatten the list
    tokens = list(flatten(tokens))
    #
    tokens = [re.sub('[^A-Za-z]+', '', str(token)) for token in tokens]

    # Mots de 3 caratères
    tokens = [token for token in tokens if len(token.lower()) > 2]

    return " ".join(tokens)

In [11]:
l = mrv_data["FABRICANT"].map(lambda x :clean_fabricant(str(x)))
fabricant_liste_cleaned = list(set(l.tolist()))

In [12]:
len(fabricant_liste_cleaned)

2280

In [8]:
new_fabricant = []
for path in os.listdir("/home/robin/Documents/DGS/data/fichiers_test"):
    try :
        data = pd.read_csv(os.path.join("/home/robin/Documents/DGS/data/fichiers_test", path))
        try :
            fabricant = data[data['Column0']=='Fabricant : Nom']['Column1'].values[0]
        except : 
            fabricant = "-1"
        new_fabricant.append(fabricant)
    except :
        print(f"Le document {path} ne peut pas être lu pour des raisons d'encodage")

In [13]:
new_fabricant_cleaned = list(map(clean_fabricant, new_fabricant))

In [9]:
new_fabricant

['arjo',
 'ZEISS',
 'Jiangsu Province Jianerkang Medical Dressing\nCo., Ltd.',
 'DORAN',
 'ROCHE',
 'SIEMENS',
 'IHT - Iberhospitex',
 'Inconnu',
 'Dr Grillo',
 'Saint-Jude Médical',
 'Diagast',
 'BAYER',
 'Medtronic',
 'LINET',
 'BIOSYNEX',
 'SMETHS MEDICAL FRANCE',
 'Pennine Healthcare',
 'NC',
 'BGI',
 'Laboratoire PAUCOURT',
 'SYNTHES',
 'LyncMed',
 'Johnson&Johnson',
 'Procare France',
 'Ethicon',
 'BAYER',
 'SMD',
 'BOSTON SCIENTIFIC',
 'Biomérieux INC',
 'SEGETEX-EIF',
 'NC',
 'ARJOHUNTLEIGH',
 'suntrine',
 'Sethygyn  EUROMEDICAL',
 'INAMED',
 'AEROCOM',
 'PEROUSE MEDICAL',
 'DORAN',
 'Allergan',
 'Biomérieux',
 'SETHYGYN',
 'Biosensors',
 'Simport',
 'BIOTRONIK',
 'BAYER',
 'GE HEALTHCARE',
 'NEURELEC',
 'Saint-Just Medical',
 'SEBBIN',
 'NOVAPLUS',
 'Cepheid',
 'LABASTIDE ROUAIROUX',
 'EVERPHARMA',
 'Saint-Jude Médical',
 'ABBOTT',
 'MEDTRONIC',
 'Saint-Jude Médical',
 'EDWARDS LIFESCIENCES',
 'COOPER vision',
 'Fresenius',
 'BBraun',
 'IOLTECH',
 'bayer',
 'CAIR LGL',
 'Hôpit

In [14]:
new_fabricant_cleaned

['arjo',
 'zeiss',
 'jiangsu province jianerkang medical dressing ltd',
 'doran',
 'roche',
 'siemens',
 'iht iberhospitex',
 'inconnu',
 'grillo',
 'saint jude medical',
 'diagast',
 'bayer',
 'medtronic',
 'linet',
 'biosynex',
 'smeths medical',
 'pennine healthcare',
 '',
 'bgi',
 'laboratoire paucourt',
 'synthes',
 'lyncmed',
 'johnson johnson',
 'procare',
 'ethicon',
 'bayer',
 'smd',
 'boston scientific',
 'biomerieux inc',
 'segetex eif',
 '',
 'arjohuntleigh',
 'suntrine',
 'sethygyn euromedical',
 'inamed',
 'aerocom',
 'perouse medical',
 'doran',
 'allergan',
 'biomerieux',
 'sethygyn',
 'biosensors',
 'simport',
 'biotronik',
 'bayer',
 'healthcare',
 'neurelec',
 'saint just medical',
 'sebbin',
 'novaplus',
 'cepheid',
 'labastide rouairoux',
 'everpharma',
 'saint jude medical',
 'abbott',
 'medtronic',
 'saint jude medical',
 'edwards lifesciences',
 'cooper vision',
 'fresenius',
 'bbraun',
 'ioltech',
 'bayer',
 'cair lgl',
 'hopital dax',
 'allmed medical gmbh',
 

In [16]:
import difflib

def compare (new_fabricant,fabricant_liste):
    for elt in new_fabricant :
        if elt.upper() in fabricant_liste:
            print("# Direct Guess:",elt)
        else :
            ratio = 0
            for fab in fabricant_liste:
                new_ratio = difflib.SequenceMatcher(lambda x: x == " ",str(elt),str(fab)).ratio()
                if new_ratio>ratio :
                    ratio = new_ratio
                    fabricant = fab
            print("Proximity Guess:",elt,ratio,fabricant)

compare(new_fabricant_cleaned,fabricant_liste_cleaned)

Proximity Guess: arjo 0.6666666666666666 arrow
Proximity Guess: zeiss 0.6 kelis
Proximity Guess: jiangsu province jianerkang medical dressing ltd 0.6382978723404256 jiangsu vedkang medical science technology ltd
Proximity Guess: doran 0.6666666666666666 orliman
Proximity Guess: roche 0.7142857142857143 roche sas
Proximity Guess: siemens 1.0 siemens
Proximity Guess: iht iberhospitex 0.8571428571428571 iberhospitex
Proximity Guess: inconnu 1.0 inconnu
Proximity Guess: grillo 0.6153846153846154 grifols
Proximity Guess: saint jude medical 1.0 saint jude medical
Proximity Guess: diagast 1.0 diagast
Proximity Guess: bayer 0.7272727272727273 baxter
Proximity Guess: medtronic 1.0 medtronic
Proximity Guess: linet 1.0 linet
Proximity Guess: biosynex 1.0 biosynex
Proximity Guess: smeths medical 0.9285714285714286 smiths medical
Proximity Guess: pennine healthcare 1.0 pennine healthcare
# Direct Guess: 
Proximity Guess: bgi 0.6666666666666666 btg
Proximity Guess: laboratoire paucourt 0.82051282051

In [32]:
print(false)

['arjo', 'Jiangsu Province Jianerkang Medical Dressing\nCo., Ltd.', 'IHT - Iberhospitex', 'NC', 'Laboratoire PAUCOURT', 'Procare France', 'NC', 'Sethygyn  EUROMEDICAL', 'INAMED', 'SETHYGYN', '-1', '-1', 'SEBBIN', 'COOPER vision', 'Fresenius', 'IOLTECH', '-1', 'Abbott', 'Allergan', 'BBraun', 'Medtronic', 'Aseptinmed']


In [45]:
new_fabricant_2 = []
for path in os.listdir("/home/robin/Documents/DGS/data/serge_csv"):
    #data = pd.read_csv(os.path.join("/home/robin/Documents/DGS/data/serge_csv", path))
    try:
        file_path = os.path.join("/home/robin/Documents/DGS/data/serge_csv", path)
        data = lib_extraction.from_csv_to_mrv_format(file_path, Colonnes = colonnes, mapping=mapping)
        fabricant = data['FABRICANT'].values[0]
        new_fabricant_2.append(fabricant)
    except :
        print(f"Le document {path} ne peut pas être lu pour des raisons d'encodage ou de format")


Le document .~lock.PS_MATERIOVIGILANCE_20201002120734040.pdf.csv# ne peut pas être lu pour des raisons d'encodage ou de format
Le document cluster_inference.json ne peut pas être lu pour des raisons d'encodage ou de format
Le document topic_inference.json ne peut pas être lu pour des raisons d'encodage ou de format
Le document mrv_format.xlsx ne peut pas être lu pour des raisons d'encodage ou de format


In [23]:
from sklearn import preprocessing
le = preprocessing.LabelBinarizer()
X = le.fit_transform(mrv_data['FABRICANT'].map(str).fillna(' '))

In [24]:
X.shape

(76954, 2566)

In [77]:
%%time
difflib.SequenceMatcher(lambda x: x == " ","arjo","arj").ratio()

CPU times: user 101 µs, sys: 0 ns, total: 101 µs
Wall time: 107 µs


0.8571428571428571